# Necessary imports and data loading

In [1]:
# necessary imports

import my_code
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.io import read_raw_edf
from mne.preprocessing import ICA

In [2]:
# loading the raw data (30 files in a list) 

path = os.getcwd()
raw_datasets = my_code.load_all_datasets(path = path)

# Pre-processing

In [3]:
# pre-processing the raw data:
# setting the reference electrodes to 'CQ_CMS', 'CQ_DRL'
# setting the montage
# high pass filter at 0.16 Hz to remove slow drifts (potentially built in in the EEG headset)
# no notch-filtering as it is built in in the EEG headset 
# annotating the raw data so that we have 5 instances of push VI, 5 instances of relax VI per dataset
# creating events from annotations 

pre_processed_datasets = my_code.preliminary_steps(raw_datasets)

# Epoch creation

In [4]:
# creating epochs from the events (starting at 0.5 s of a push/relax instance and ending at 9.5 s of a push/relax instance)

# first, creating a set of epochs based just on one dataset

events_from_annot, event_dict = mne.events_from_annotations(pre_processed_datasets[0])
event_dict = {"Push" : 1, "Relax" : 2}
baseline = (0.5, 0.5)
delay = 0.5
epochs_all = mne.Epochs(pre_processed_datasets[0], events=events_from_annot, event_id = event_dict, baseline = baseline, tmin = 0.5, tmax = (10-delay), preload = True, reject_by_annotation=False)

# then, looping it so more sets of epochs are appended 

epochs_all = my_code.create_epochs(pre_processed_datasets=pre_processed_datasets)

IndexError: list index out of range

# Applying different cleaning & artifact detection methods

#### ICA --> peak to peak 

In [ ]:
# artifact detection/repair/deletion mix 1
# first ICA; visualize the components

my_code.apply_ica(epochs_all)

    # decide which components to reject and reject them
    
epochs_ica = epochs_all.copy()
ica = ICA(n_components=14, random_state=97)
ica.fit(epochs_ica)
ica.exclude = [1,3]
epochs_ica = ica.apply(epochs_ica)

# then peak-to-peak amplitude rejection threshold of 200 mV

reject_criteria = dict(eeg=200e-6)

epochs_ica_ptp_200 = epochs_ica.drop_bad(reject=reject_criteria)
print(epochs_ica_ptp_200.drop_log)
epochs_ica_ptp_200.plot_drop_log()

# equalizing instance counts

epochs_ica_ptp_200.equalize_event_counts(epochs_ica_ptp_200.event_id)

#### peak to peak --> ICA 

In [ ]:
# artifact detection/repair/deletion mix 2

# first peak-to-peak amplitude rejection threshold of 150 mV
epochs_ptp_200 = epochs_all.copy()
reject_criteria = dict(eeg=200e-6)

epochs_ptp_200 = epochs_ptp_200.drop_bad(reject=reject_criteria)
print(epochs_ptp_200.drop_log)
epochs_ptp_200.plot_drop_log()

# then ICA; visualize the components

my_code.apply_ica(epochs_ptp_200)

    # decide which components to reject and reject them
    
ica = ICA(n_components=14, random_state=97)
ica.fit(epochs_ptp_200)
ica.exclude = [1,3]
epochs_ptp_200_ica = ica.apply(epochs_ptp_200)

# equalizing instance counts

epochs_ptp_200_ica.equalize_event_counts(epochs_ptp_200_ica.event_id)


#### ICA --> Autoreject

In [ ]:
# artifact detection/repair/deletion mix 3


# first ICA; visualize the components

#my_code.apply_ica(epochs_all)

    # decide which components to reject and reject them
    
epochs_ica = epochs_all.copy()
ica = ICA(n_components=14, random_state=97)
ica.fit(epochs_ica)
ica.exclude = [1,3]
epochs_ica = ica.apply(epochs_ica)

# then Autoreject

epochs_ica_autoreject = my_code.clean_epochs(epochs_all = epochs_ica)

# equalizing instance counts

epochs_ica_autoreject.equalize_event_counts(epochs_ica_autoreject.event_id)

#### Autoreject --> ICA 

In [ ]:
# artifact detection/repair/deletion mix 4

# first Autoreject

epochs_autoreject = epochs_all.copy()
epochs_autoreject = my_code.clean_epochs(epochs_all = epochs_autoreject)


# then ICA; visualize the components

#my_code.apply_ica(epochs_autoreject)

    # decide which components to reject and reject them
    
ica = ICA(n_components=14, random_state=97)
ica.fit(epochs_autoreject)
ica.exclude = [1,3]
epochs_autoreject_ica = ica.apply(epochs_autoreject)

# equalizing instance counts

epochs_autoreject_ica.equalize_event_counts(epochs_autoreject_ica.event_id)


# Feature extraction methods

## Time-frequency

In [ ]:
alpha_push_power, alpha_relax_power = my_code.create_TF_object_for_plotting(8, 12, 20, epochs_ica_ptp_200["Push"], epochs_ica_ptp_200["Relax"])
beta_push_power, beta_relax_power = my_code.create_TF_object_for_plotting(12, 30, 20, epochs_ica_ptp_200["Push"], epochs_ica_ptp_200["Relax"])
gamma_push_power, gamma_relax_power = my_code.create_TF_object_for_plotting(30, 80, 20, epochs_ica_ptp_200["Push"], epochs_ica_ptp_200["Relax"])

In [ ]:
# extract power values over time across different frequencies
# to be able to plot topomaps 
freqs = np.logspace(*np.log10([8, 12]), num=20)
n_cycles = freqs / 2.  # different number of cycle per frequency

alpha_push_power, itc = mne.time_frequency.tfr_morlet(epochs_ica_ptp_200["Push"], freqs=freqs, n_cycles=n_cycles, use_fft=True,
                                           return_itc=True, decim=3, n_jobs=1)
alpha_relax_power, itc = mne.time_frequency.tfr_morlet(epochs_ica_ptp_200["Relax"], freqs=freqs, n_cycles=n_cycles, use_fft=True,
                                           return_itc=True, decim=3, n_jobs=1)

freqs = np.logspace(*np.log10([12, 30]), num=20)
n_cycles = freqs / 2.  # different number of cycle per frequency

beta_push_power, itc = mne.time_frequency.tfr_morlet(epochs_ica_ptp_200["Push"], freqs=freqs, n_cycles=n_cycles, use_fft=True,
                                           return_itc=True, decim=3, n_jobs=1)
beta_relax_power, itc = mne.time_frequency.tfr_morlet(epochs_ica_ptp_200["Relax"], freqs=freqs, n_cycles=n_cycles, use_fft=True,
                                           return_itc=True, decim=3, n_jobs=1)

freqs = np.logspace(*np.log10([30, 80]), num=20)
n_cycles = freqs / 2.  # different number of cycle per frequency

gamma_push_power, itc = mne.time_frequency.tfr_morlet(epochs_ica_ptp_200["Push"], freqs=freqs, n_cycles=n_cycles, use_fft=True,
                                           return_itc=True, decim=3, n_jobs=1)
gamma_relax_power, itc = mne.time_frequency.tfr_morlet(epochs_ica_ptp_200["Relax"], freqs=freqs, n_cycles=n_cycles, use_fft=True,
                                           return_itc=True, decim=3, n_jobs=1)

## PyEEG features

In [ ]:
# extracting detrended fluctuation analysis coefficients, fisher information, hurst component
# hjorth fractal dimension, hjirth mobility, hjorth complexity, petrosian fractal dimension

In [ ]:
fi_epochs_over_channels_push, dfa_epochs_over_channels_push, he_epochs_over_channels_push, hfd_epochs_over_channels_push, hm_epochs_over_channels_push, hc_epochs_over_channels_push, pfd_epochs_over_channels_push, fi_epochs_over_channels_push, dfa_epochs_over_channels_push, he_epochs_over_channels_push, hfd_epochs_over_channels_push, hm_epochs_over_channels_push, hc_epochs_over_channels_push, pfd_epochs_over_channels_push, all_pyeeg_features = my_code.extract_pyeegfeatures(115, epochs_ica_ptp_200["Push"],epochs_ica_ptp_200["Relax"])  ) 

## Common Spatial Patterns

In [ ]:
# introductory explanation

## Principle Component Analysis

In [ ]:
# introductory explanation

## RGB values from spectrogram images

In [ ]:
# introductory explanation

# Data exploration informing feature-engineering

In [ ]:
print(epochs_ica_ptp_200)
print(epochs_ptp_200_ica)
print(epochs_ica_autoreject)
print(epochs_autoreject_ica)

In [ ]:
# visually inspecting the epochs: plotting their averages 
# need to update it for all the artifact rejection mixes

push_average = epochs_ica_ptp_200["Push"].average()
relax_average = epochs_ica_ptp_200["Relax"].average()
mne.viz.plot_compare_evokeds([push_average, relax_average])

In [ ]:
my_code.plot_topomaps(alpha_push_power, alpha_relax_power,  beta_push_power, beta_relax_power,  gamma_push_power,  gamma_relax_power)

In [ ]:
# more time-frequency plotting
# out of the pop up graphs we can select the channels we want to put into spectograms
# power will be averaged over the selected channels 

# TO MODULARIZE THE CODE

baseline=(0.5,0.5)

alpha_push_power.plot_topomap(ch_type='eeg', tmin=0.5, tmax=9.5, fmin=8, fmax=12,
                   baseline=baseline,
                   title='Push Alpha', show=False, contours=1)
alpha_relax_power.plot_topomap(ch_type='eeg', tmin=0.5, tmax=9.5, fmin=8, fmax=12,
                   baseline=baseline,
                   title='Relax Alpha', show=False, contours=1)

beta_push_power.plot_topomap(ch_type='eeg', tmin=0.5, tmax=9.5, fmin=12, fmax=30,
                   baseline=baseline,
                   title='Push Beta', show=False, contours=1)
beta_relax_power.plot_topomap(ch_type='eeg', tmin=0.5, tmax=9.5, fmin=12, fmax=30,
                   baseline=baseline,
                   title='Relax Beta', show=False, contours=1)

gamma_push_power.plot_topomap(ch_type='eeg',tmin=0.5, tmax=9.5, fmin=30, fmax=80,
                   baseline=baseline,
                   title='Push Gamma', show=False, contours=1)
gamma_relax_power.plot_topomap(ch_type='eeg', tmin=0.5, tmax=9.5, fmin=30, fmax=80,
                   baseline=baseline,
                   title='Relax Gamma', show=False, contours=1)

In [ ]:
# taking averages of frequency bands, so that we have vectors that represent epoch x channel x time per alpha, beta, gamma
# subtracting push frequencies per frequency from relax frequencies per frequency
# dividing an epoch into beginning, middle, end
# taking average of those differences per beginning, middle, end to visualize where biggest frequency differences lie

# TO MODULARIZE THE CODE

power_push_alpha_data = alpha_push_power.data
power_push_alpha_data_average = power_push_alpha_data.mean(axis=1)
power_relax_alpha_data = alpha_relax_power.data
power_relax_alpha_data_average = power_relax_alpha_data.mean(axis=1)

difference_AF3_alpha = power_push_alpha_data_average[0] - power_relax_alpha_data_average[0]
difference_AF4_alpha = power_push_alpha_data_average[13] - power_relax_alpha_data_average[13]
difference_F7_alpha = power_push_alpha_data_average[1] - power_relax_alpha_data_average[1]
difference_F8_alpha = power_push_alpha_data_average[12] - power_relax_alpha_data_average[12]
difference_F3_alpha = power_push_alpha_data_average[2] - power_relax_alpha_data_average[2]
difference_F4_alpha = power_push_alpha_data_average[11] - power_relax_alpha_data_average[11]
difference_FC5_alpha = power_push_alpha_data_average[3] - power_relax_alpha_data_average[3]
difference_FC6_alpha = power_push_alpha_data_average[10] - power_relax_alpha_data_average[10]
difference_T7_alpha = power_push_alpha_data_average[4] - power_relax_alpha_data_average[4]
difference_T8_alpha = power_push_alpha_data_average[9] - power_relax_alpha_data_average[9]
difference_P7_alpha = power_push_alpha_data_average[5] - power_relax_alpha_data_average[5]
difference_P8_alpha = power_push_alpha_data_average[8] - power_relax_alpha_data_average[8]
difference_O1_alpha = power_push_alpha_data_average[6] - power_relax_alpha_data_average[6]
difference_O2_alpha = power_push_alpha_data_average[7] - power_relax_alpha_data_average[7]

power_push_beta_data = beta_push_power.data
power_push_beta_data_average = power_push_beta_data.mean(axis=1)
power_relax_beta_data = beta_relax_power.data
power_relax_beta_data_average = power_relax_beta_data.mean(axis=1)

difference_AF3_beta = power_push_beta_data_average[0] - power_relax_beta_data_average[0]
difference_AF4_beta = power_push_beta_data_average[13] - power_relax_beta_data_average[13]
difference_F7_beta = power_push_beta_data_average[1] - power_relax_beta_data_average[1]
difference_F8_beta = power_push_beta_data_average[12] - power_relax_beta_data_average[12]
difference_F3_beta = power_push_beta_data_average[2] - power_relax_beta_data_average[2]
difference_F4_beta = power_push_beta_data_average[11] - power_relax_beta_data_average[11]
difference_FC5_beta = power_push_beta_data_average[3] - power_relax_beta_data_average[3]
difference_FC6_beta = power_push_beta_data_average[10] - power_relax_beta_data_average[10]
difference_T7_beta = power_push_beta_data_average[4] - power_relax_beta_data_average[4]
difference_T8_beta = power_push_beta_data_average[9] - power_relax_beta_data_average[9]
difference_P7_beta = power_push_beta_data_average[5] - power_relax_beta_data_average[5]
difference_P8_beta = power_push_beta_data_average[8] - power_relax_beta_data_average[8]
difference_O1_beta = power_push_beta_data_average[6] - power_relax_beta_data_average[6]
difference_O2_beta = power_push_beta_data_average[7] - power_relax_beta_data_average[7]

power_push_gamma_data = gamma_push_power.data
power_push_gamma_data_average = power_push_gamma_data.mean(axis=1)
power_relax_gamma_data = gamma_relax_power.data
power_relax_gamma_data_average = power_relax_gamma_data.mean(axis=1)

difference_AF3_gamma = power_push_gamma_data_average[0] - power_relax_gamma_data_average[0]
difference_AF4_gamma = power_push_gamma_data_average[13] - power_relax_gamma_data_average[13]
difference_F7_gamma = power_push_gamma_data_average[1] - power_relax_gamma_data_average[1]
difference_F8_gamma = power_push_gamma_data_average[12] - power_relax_gamma_data_average[12]
difference_F3_gamma = power_push_gamma_data_average[2] - power_relax_gamma_data_average[2]
difference_F4_gamma = power_push_gamma_data_average[11] - power_relax_gamma_data_average[11]
difference_FC5_gamma = power_push_gamma_data_average[3] - power_relax_gamma_data_average[3]
difference_FC6_gamma= power_push_gamma_data_average[10] - power_relax_gamma_data_average[10]
difference_T7_gamma = power_push_gamma_data_average[4] - power_relax_gamma_data_average[4]
difference_T8_gamma = power_push_gamma_data_average[9] - power_relax_gamma_data_average[9]
difference_P7_gamma = power_push_gamma_data_average[5] - power_relax_gamma_data_average[5]
difference_P8_gamma = power_push_gamma_data_average[8] - power_relax_gamma_data_average[8]
difference_O1_gamma = power_push_gamma_data_average[6] - power_relax_gamma_data_average[6]
difference_O2_gamma= power_push_gamma_data_average[7] - power_relax_gamma_data_average[7]


#

##### average differences for channels and frequencies and times 
# i will divide all the time points in 3 parts, part I, part II, part III

# list of difference values we have 

alpha_differences = [difference_AF3_alpha, difference_AF4_alpha, difference_F7_alpha, difference_F8_alpha, difference_F3_alpha, difference_F4_alpha, difference_FC5_alpha, difference_FC6_alpha, difference_T7_alpha, difference_T8_alpha, difference_P7_alpha, difference_P8_alpha, difference_O1_alpha, difference_O2_alpha]


# allpha differences part I 

alpha1_differences_values = []

for channel in alpha_differences:
    value = abs(channel[slice(0,199)].mean(axis=0))
    alpha1_differences_values.append(value)
    
# allpha differences part II

alpha2_differences_values = []

for channel in alpha_differences:
    value = abs(channel[slice(199,2*199)].mean(axis=0))
    alpha2_differences_values.append(value)
    
# allpha differences part III

alpha3_differences_values = []

for channel in alpha_differences:
    value = abs(channel[slice(2*199,3*199)].mean(axis=0))
    alpha3_differences_values.append(value)


beta_differences = [difference_AF3_beta, difference_AF4_beta, difference_F7_beta, difference_F8_beta, difference_F3_beta, difference_F4_beta, difference_FC5_beta, difference_FC6_beta, difference_T7_beta, difference_T8_beta, difference_P7_beta, difference_P8_beta, difference_O1_beta, difference_O2_beta]


# allpha differences part I 

beta1_differences_values = []

for channel in beta_differences:
    value = abs(channel[slice(0,199)].mean(axis=0))
    beta1_differences_values.append(value)
    
# allpha differences part II

beta2_differences_values = []

for channel in beta_differences:
    value = abs(channel[slice(199,2*199)].mean(axis=0))
    beta2_differences_values.append(value)
    
# allpha differences part III

beta3_differences_values = []

for channel in beta_differences:
    value = abs(channel[slice(2*199,3*199)].mean(axis=0))
    beta3_differences_values.append(value)


    
gamma_differences = [difference_AF3_gamma, difference_AF4_gamma, difference_F7_gamma, difference_F8_gamma, difference_F3_gamma, difference_F4_gamma, difference_FC5_gamma, difference_FC6_gamma, difference_T7_gamma, difference_T8_gamma, difference_P7_gamma, difference_P8_gamma, difference_O1_gamma, difference_O2_gamma]


# allpha differences part I 

gamma1_differences_values = []

for channel in gamma_differences:
    value = abs(channel[slice(0,199)].mean(axis=0))
    gamma1_differences_values.append(value)
    
# allpha differences part II

gamma2_differences_values = []

for channel in gamma_differences:
    value = abs(channel[slice(199,2*199)].mean(axis=0))
    gamma2_differences_values.append(value)
    
# allpha differences part III

gamma3_differences_values = []

for channel in gamma_differences:
    value = abs(channel[slice(2*199,3*199)].mean(axis=0))
    gamma3_differences_values.append(value)

print("Alpha differences values in 1st chunk:\n", alpha1_differences_values)
print("Alpha differences values in 2nd chunk:\n", alpha2_differences_values)
print("Alpha differences values in 3rd chunk:\n", alpha3_differences_values)
print("Beta differences values in 1st chunk:\n", beta1_differences_values)
print("Beta differences values in 2nd chunk:\n", beta2_differences_values)
print("Beta differences values in 3rd chunk:\n", beta3_differences_values)
print("Gamma differences values in 1st chunk:\n", gamma1_differences_values)
print("Gamma differences values in 2nd chunk:\n", gamma2_differences_values)
print("Gamma differences values in 3rd chunk:\n", gamma3_differences_values)

plt.figure(85)
plt.plot(epochs_all.ch_names,alpha1_differences_values, label = "alpha")
plt.plot(epochs_all.ch_names,beta1_differences_values, label = "beta")
plt.plot(epochs_all.ch_names,gamma1_differences_values, label = "gamma")
plt.xlabel("Channel")
plt.ylabel("Power difference")
plt.suptitle('Average differences in push/relax in 1 s - 3.34 s ', fontsize=16)
plt.legend()
plt.show

plt.figure(86)
plt.plot(epochs_all.ch_names,alpha2_differences_values, label = "alpha")
plt.plot(epochs_all.ch_names,beta2_differences_values, label = "beta")
plt.plot(epochs_all.ch_names,gamma2_differences_values, label = "gamma")
plt.xlabel("Channel")
plt.ylabel("Power difference")
plt.suptitle('Average differences in push/relax in 3.34 s - 5.67 s ', fontsize=16)
plt.legend()
plt.show

plt.figure(87)
plt.plot(epochs_all.ch_names,alpha3_differences_values, label = "alpha")
plt.plot(epochs_all.ch_names,beta3_differences_values, label = "beta")
plt.plot(epochs_all.ch_names,gamma3_differences_values, label = "gamma")
plt.xlabel("Channel")
plt.ylabel("Power difference")
plt.suptitle('Average differences in push/relax in 5.67 s - 8s ', fontsize=16)
plt.legend()
plt.show



In [ ]:
# running Sliding estimator on raw data and cropping epochs based on that 

In [ ]:
# extracting alpha, beta, gamma frequencies and their mixes per channel over time 
# repeat for every artifact rejection mix 

freqs, alpha_push_power, alpha_push_itc, instances = my_code.create_time_frequency_matrices(8, 12, 20, 115, 14, 769, epochs_ica_ptp_200["Push"])
freqs, alpha_relax_power, alpha_relax_itc, instances = my_code.create_time_frequency_matrices(8, 12, 20, 115, 14, 769, epochs_ica_ptp_200["Relax"])
freqs, beta_push_power, beta_push_itc, instances = my_code.create_time_frequency_matrices(13, 30, 20, 115, 14, 769, epochs_ica_ptp_200["Push"])
freqs, beta_relax_power, beta_relax_itc, instances = my_code.create_time_frequency_matrices(13, 30, 20, 115, 14, 769, epochs_ica_ptp_200["Relax"])
freqs, gamma_push_power, gamma_push_itc, instances = my_code.create_time_frequency_matrices(30, 80, 20, 115, 14, 769, epochs_ica_ptp_200["Push"])
freqs, gamma_relax_power, gamma_relax_itc, instances = my_code.create_time_frequency_matrices(30, 80, 20, 115, 14, 769, epochs_ica_ptp_200["Relax"])
alpha_beta_push_power = np.concatenate((alpha_push_power, beta_push_power), axis=3)
alpha_beta_relax_power = np.concatenate((alpha_relax_power, beta_relax_power), axis=3)
alpha_beta_gamma_push_power = np.concatenate((alpha_beta_push_power, gamma_push_power), axis=3)
alpha_beta_gamma_relax_power = np.concatenate((alpha_beta_relax_power, gamma_relax_power), axis=3)

In [ ]:
# running a series of SlidingEstimators to see where the different frequencies achieve biggest accuracy

alpha_push_power_channels, alpha_relax_power_channels, mean_scores_alpha = my_code.sliding_estimator(alpha_push_power, alpha_relax_power, 115, 769)
beta_push_power_channels, beta_relax_power_channels, mean_scores_beta = my_code.sliding_estimator(beta_push_power, beta_relax_power, 115, 769)
gamma_push_power_channels, gamma_relax_power_channels, mean_scores_gamma = my_code.sliding_estimator(gamma_push_power, gamma_relax_power, 115, 769)

# repeat for every artifact rejection mix 

In [ ]:
# cropping data into different times & channels based on SlidingEstimator and visual inspection of my method

# based on LDA on frequency data from all epoch (choosing timepoints that gave SE an accuracy over a certain threshold)

res_alpha = []
for idx in range(0, len(mean_scores_alpha)) :
    if mean_scores_alpha[idx] > 0.60:
        res_alpha.append(idx)

samples_alpha_push = alpha_push_power_channels[:,:,res_alpha]
samples_alpha_relax = alpha_push_power_channels[:,:,res_alpha]

# based on my own visual inspection

    #visualinspection_alpha_push = alpha_push_power_channels[:,:,my_timepoints]
    #visualinspection_alpha_relax = alpha_push_power_channels[:,:,my_timepoints]

# cropping channels based on my own visual inspection 

    # e.g. choosing only O2, F8, F7, AF4, P7 for alpha frequencies; channel indices: O2 - 7, F8 - 12, F7 - 1, AF3 - 13, P7 - 5
    

alpha_push_power_croppedchannels = my_code.select_channels((7,12,1,13,5), alpha_push_power, 115, 769)
alpha_relax_power_croppedchannels = my_code.select_channels((7,12,1,13,5), alpha_relax_power, 115, 769)

In [ ]:
# classifying different combinations of the TF data by LDA, SVM and DL architectures

# alpha frequencies from different artifact mixes 

    # e.g. 

alpha_mix1_data = my_code.run_lda(alpha_push_power, alpha_relax_power, 115)
alpha_mix1_data = my_code.run_svm(alpha_push_power, alpha_relax_power, 115)

# beta frequencies from different artifact mixes 

    # e.g. 

beta_mix1_data = my_code.run_lda(beta_push_power, beta_relax_power, 115)
beta_mix1_data = my_code.run_svm(beta_push_power, beta_relax_power, 115)

# gamma frequencies from different artifact mixes 

    # e.g. 

gamma_mix1_data = my_code.run_lda(gamma_push_power, gamma_relax_power, 115)
gamma_mix1_data = my_code.run_svm(gamma_push_power, gamma_relax_power, 115)

# alpha&beta frequencies from different artifact mixes 

    # e.g. 

alpha_beta_mix1_data = my_code.run_lda(alpha_beta_push_power, alpha_beta_relax_power, 115)
alpha_beta_mix1_data = my_code.run_svm(alpha_beta_push_power, alpha_beta_relax_power, 115)

# alpha, beta&gamma frequencies from different artifact mixes 

    # e.g. 

alpha_beta_gamma_mix1_data = my_code.run_lda(alpha_beta_gamma_push_power, alpha_beta_gamma_relax_power, 115)
alpha_beta_gamma_mix1_data = my_code.run_svm(alpha_beta_gamma_push_power, alpha_beta_gamma_relax_power, 115)



In [ ]:
# classification of alpha frequencies at timpoints where SE accuracy was higher than 65%

alpha_samples_mix1_data = my_code.run_lda(samples_alpha_push, samples_alpha_relax, 115)
alpha_samples_mix1_data = my_code.run_svm(samples_alpha_push, samples_alpha_relax, 115)

# classification of alpha frequencies from different channels based on visual inspection 

alpha_cropped_channels_mix1_data = my_code.run_lda(alpha_push_power_croppedchannels, alpha_relax_power_croppedchannels, 115)
alpha_cropped_channels_mix1_data = my_code.run_svm(alpha_push_power_croppedchannels, alpha_relax_power_croppedchannels, 115)


# repeat those combination for the rest of frequency bands and the rest of artifact rejection mixes 


In [ ]:
# using pyEEG module to extract the features
# TO MODULARIZE THE CODE


import pyeeg

channels_list = np.arange(14)
channels_list
instances = np.arange(115)
print(instances.shape)

push_epochs_data = epochs_all["Push"]
relax_epochs_data = epochs_all["Relax"]

push_epochs_data = push_epochs_data.get_data()
relax_epochs_data = relax_epochs_data.get_data()

labels = np.concatenate((np.ones(shape=(115)), np.zeros(shape=(115))), axis = 0)


#### Extracting DFA parameters

dfa_per_channel_push = np.zeros(shape=(14))
dfa_epochs_over_channels_push = np.zeros(shape=(115, 14))
dfa_per_channel_relax = np.zeros(shape=(14))
dfa_epochs_over_channels_relax = np.zeros(shape=(115, 14))

for instance in instances:
    
    for channel in channels_list:
        dfa_push = pyeeg.dfa(push_epochs_data[instance, channel])
        dfa_per_channel_push[channel]=dfa_push
        dfa_relax = pyeeg.dfa(relax_epochs_data[instance, channel])
        dfa_per_channel_relax[channel]=dfa_relax
    
    dfa_epochs_over_channels_push[instance] = dfa_per_channel_push
    dfa_epochs_over_channels_relax[instance] = dfa_per_channel_relax

print(dfa_epochs_over_channels_push.shape)
print(dfa_epochs_over_channels_relax.shape)

#### Extracting Fisher Information

fi_per_channel_push = np.zeros(shape=(14))
fi_epochs_over_channels_push = np.zeros(shape=(115, 14))
fi_per_channel_relax = np.zeros(shape=(14))
fi_epochs_over_channels_relax = np.zeros(shape=(115, 14))

for instance in instances:
    
    for channel in channels_list:
        fi_push = pyeeg.fisher_info(push_epochs_data[instance, channel],1, 2)
        fi_per_channel_push[channel]=fi_push
        fi_relax = pyeeg.fisher_info(relax_epochs_data[instance, channel], 1,2)
        fi_per_channel_relax[channel]=fi_relax
    
    fi_epochs_over_channels_push[instance] =  fi_per_channel_push
    fi_epochs_over_channels_relax[instance] =  fi_per_channel_relax

print(fi_epochs_over_channels_push.shape)
print(fi_epochs_over_channels_relax.shape)

#### Extracting Hurst Exponent

pyeeg.hurst(push_epochs_data[6, 4])

he_per_channel_push = np.zeros(shape=(14))
he_epochs_over_channels_push = np.zeros(shape=(115, 14))
he_per_channel_relax = np.zeros(shape=(14))
he_epochs_over_channels_relax = np.zeros(shape=(115, 14))

for instance in instances:
    
    for channel in channels_list:
        he_push = pyeeg.hurst(push_epochs_data[instance, channel])
        he_per_channel_push[channel]=he_push
        he_relax = pyeeg.hurst(push_epochs_data[instance, channel])
        he_per_channel_relax[channel]=he_relax
    
    he_epochs_over_channels_push[instance] =  he_per_channel_push
    he_epochs_over_channels_relax[instance] =  he_per_channel_relax

print(he_epochs_over_channels_push.shape)
print(he_epochs_over_channels_relax.shape)

#### Extracting Hjorth Fractal Dimension 

### according to a paper saved in my feature extraction folder, value of 18 is best for kmax argument of hfd

pyeeg.hfd(push_epochs_data[6, 4],18)

hfd_per_channel_push = np.zeros(shape=(14))
hfd_epochs_over_channels_push = np.zeros(shape=(115, 14))
hfd_per_channel_relax = np.zeros(shape=(14))
hfd_epochs_over_channels_relax = np.zeros(shape=(115, 14))

for instance in instances:
    
    for channel in channels_list:
        hfd_push = pyeeg.hfd(push_epochs_data[instance, channel], 18)
        hfd_per_channel_push[channel]=hfd_push
        hfd_relax = pyeeg.hfd(push_epochs_data[instance, channel], 18)
        hfd_per_channel_relax[channel]=hfd_relax
    
    hfd_epochs_over_channels_push[instance] =  hfd_per_channel_push
    hfd_epochs_over_channels_relax[instance] =  hfd_per_channel_relax

print(hfd_epochs_over_channels_push.shape)
print(hfd_epochs_over_channels_relax.shape)

#### Extracting Hjorth Mobility & Complexity


# first is mobility
# second is complexity

hm_per_channel_push = np.zeros(shape=(14))
hm_epochs_over_channels_push = np.zeros(shape=(115, 14))
hc_per_channel_push = np.zeros(shape=(14))
hc_epochs_over_channels_push = np.zeros(shape=(115, 14))
hm_per_channel_relax = np.zeros(shape=(14))
hm_epochs_over_channels_relax = np.zeros(shape=(115, 14))
hc_per_channel_relax = np.zeros(shape=(14))
hc_epochs_over_channels_relax = np.zeros(shape=(115, 14))

for instance in instances:
    
    for channel in channels_list:
        hmc_push = pyeeg.hjorth(push_epochs_data[instance, channel])
        hm_per_channel_push[channel]=hmc_push[0]
        hc_per_channel_push[channel]=hmc_push[1]
        hmc_relax = pyeeg.hjorth(push_epochs_data[instance, channel])
        hm_per_channel_relax[channel]=hmc_relax[0]
        hc_per_channel_relax[channel]=hmc_relax[1]
    
    hm_epochs_over_channels_push[instance] =  hm_per_channel_push
    hc_epochs_over_channels_push[instance] =  hc_per_channel_push
    hm_epochs_over_channels_relax[instance] =  hm_per_channel_relax
    hc_epochs_over_channels_relax[instance] =  hc_per_channel_relax

print(hm_epochs_over_channels_push.shape)
print(hc_epochs_over_channels_relax.shape)
print(hm_epochs_over_channels_push.shape)
print(hc_epochs_over_channels_relax.shape)

#### Extracting Petrosian Fractal Dimension


pfd_per_channel_push = np.zeros(shape=(14))
pfd_epochs_over_channels_push = np.zeros(shape=(115, 14))
pfd_per_channel_relax = np.zeros(shape=(14))
pfd_epochs_over_channels_relax = np.zeros(shape=(115, 14))

for instance in instances:
    
    for channel in channels_list:
        pfd_push = pyeeg.pfd(push_epochs_data[instance, channel])
        pfd_per_channel_push[channel]=pfd_push
        pfd_relax = pyeeg.pfd(push_epochs_data[instance, channel])
        pfd_per_channel_relax[channel]=pfd_relax
    
    pfd_epochs_over_channels_push[instance] =  pfd_per_channel_push
    pfd_epochs_over_channels_relax[instance] =  pfd_per_channel_relax

print(pfd_epochs_over_channels_push.shape)
print(pfd_epochs_over_channels_relax.shape)

In [ ]:
# classifying different combinations of the pyEEG features by LDA, SVM and DL architectures

### Running those PyEEG features through LDA

    from sklearn.pipeline import Pipeline
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from sklearn.model_selection import ShuffleSplit, cross_val_score

    scores = []
    all_data = all_pyeeg_features
    all_data_train = all_pyeeg_features.copy()
    cv = ShuffleSplit(10, test_size=0.2, random_state=42)
    cv_split = cv.split(all_data_train)

# Assemble a classifier
    lda = LinearDiscriminantAnalysis()

# Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('LDA', lda)])
    scores = cross_val_score(clf, all_data_train, labels, cv=cv, n_jobs=1)

# Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))


### Running feature-by-feature through LDA 

fi_epochs_over_channels = np.concatenate((fi_epochs_over_channels_push, fi_epochs_over_channels_relax), axis = 0)
dfa_epochs_over_channels = np.concatenate((dfa_epochs_over_channels_push, dfa_epochs_over_channels_relax), axis = 0)
he_epochs_over_channels = np.concatenate((he_epochs_over_channels_push, he_epochs_over_channels_relax), axis = 0)
hfd_epochs_over_channels = np.concatenate((hfd_epochs_over_channels_push, hfd_epochs_over_channels_relax), axis = 0)
hm_epochs_over_channels = np.concatenate((hm_epochs_over_channels_push, hm_epochs_over_channels_relax), axis = 0)
hc_epochs_over_channels = np.concatenate((hc_epochs_over_channels_push, hc_epochs_over_channels_relax), axis = 0)
pfd_epochs_over_channels = np.concatenate((pfd_epochs_over_channels_push, pfd_epochs_over_channels_relax), axis = 0)
features_list = fi_epochs_over_channels, dfa_epochs_over_channels, he_epochs_over_channels, hfd_epochs_over_channels, hm_epochs_over_channels, hc_epochs_over_channels, pfd_epochs_over_channels

for item in features_list:     
    
    push = np.ones(shape=(121))
    relax = np.zeros(shape=(121))
    labels = np.concatenate((push ,relax), axis = 0)
    
    
    # Now, we run the alpha data through LDA

    from sklearn.pipeline import Pipeline
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from sklearn.model_selection import ShuffleSplit, cross_val_score

    scores = []
    all_data = item
    all_data_train = item.copy()
    cv = ShuffleSplit(10, test_size=0.2, random_state=42)
    cv_split = cv.split(all_data_train)

# Assemble a classifier
    lda = LinearDiscriminantAnalysis()

# Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('LDA', lda)])
    scores = cross_val_score(clf, all_data_train, labels, cv=cv, n_jobs=1)

# Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))
    
    

    
### Running those PyEEG features through SVM

In [ ]:
# carrying out Principal Comonent Analysis (PCA) to extract features

# TO FINISH

In [ ]:
# classifying PCA features by LDA, SVM and DL architectures

# TO FINISH

In [ ]:
# carrying out Common Spatial Patterns (CSP) to extract features & classifying CSP features by LDA, SVM and DL architectures

In [ ]:
scores = []

# just alpha
alpha_score = my_code.csp_lda(alpha_push_power, alpha_relax_power, 121)
# just beta
beta_score = my_code.csp_lda(beta_push_power, beta_relax_power, 121)
# just gamma
gamma_score = my_code.csp_lda(gamma_push_power, gamma_relax_power, 121)
# alpha & beta
alpha_beta_score = my_code.csp_lda(alpha_beta_push_power, alpha_beta_relax_power, 121)

# alpha, beta & gamma

alpha_beta_gamma_score = my_code.csp_lda(alpha_beta_gamma_push_power, alpha_beta_gamma_relax_power, 121)




In [ ]:
# classifying combinations of TF, PyEEG, PCA & CSP features by LDA, SVM and DL architectures

In [ ]:
alpha_powers = np.concatenate((alpha_push_power, alpha_push_power), axis = 0)
alpha_powers.shape

alpha_powers=alpha_powers.mean(axis=2)

alpha_powers.shape

alpha_powers = alpha_powers.reshape(242, -1)

alpha_powers.shape

alpha_fi_vectors = np.concatenate((alpha_powers, fi_epochs_over_channels), axis = 1)
alpha_fi_vectors.shape

 
    
    push = np.ones(shape=(121))
    relax = np.zeros(shape=(121))
    labels = np.concatenate((push ,relax), axis = 0)
    
    
    # Now, we run the alpha data through LDA

    from sklearn.pipeline import Pipeline
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from sklearn.model_selection import ShuffleSplit, cross_val_score

    scores = []
    all_data = alpha_fi_vectors
    all_data_train = alpha_fi_vectors.copy()
    cv = ShuffleSplit(10, test_size=0.2, random_state=42)
    cv_split = cv.split(all_data_train)

# Assemble a classifier
    lda = LinearDiscriminantAnalysis()

# Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('LDA', lda)])
    scores = cross_val_score(clf, all_data_train, labels, cv=cv, n_jobs=1)

# Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))
    
    


### Combining Fisher Information with gamma vectors

gamma_powers = np.concatenate((gamma_push_power, gamma_push_power), axis = 0)
gamma_powers.shape

gamma_powers=gamma_powers.mean(axis=2)

gamma_powers.shape

gamma_powers = gamma_powers.reshape(242, -1)

gamma_powers.shape

gamma_fi_vectors = np.concatenate((gamma_powers, fi_epochs_over_channels), axis = 1)
gamma_fi_vectors.shape

 
    
    push = np.ones(shape=(121))
    relax = np.zeros(shape=(121))
    labels = np.concatenate((push ,relax), axis = 0)
    
    
    # Now, we run the alpha data through LDA

    from sklearn.pipeline import Pipeline
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from sklearn.model_selection import ShuffleSplit, cross_val_score

    scores = []
    all_data = alpha_fi_vectors
    all_data_train = gamma_fi_vectors.copy()
    cv = ShuffleSplit(10, test_size=0.2, random_state=42)
    cv_split = cv.split(all_data_train)

# Assemble a classifier
    lda = LinearDiscriminantAnalysis()

# Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('LDA', lda)])
    scores = cross_val_score(clf, all_data_train, labels, cv=cv, n_jobs=1)

# Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))
    
    




In [ ]:
# classifying raw data & spectograms by DL architectures 

# TO FINISH

In [ ]:
# comparisons section

# TO START